In [1]:
## IMPORTS

In [41]:
import pandas as pd
import numpy as np
from scipy import interpolate
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import random
from sklearn.model_selection import train_test_split
from collections import Counter

In [2]:
## LOAD CSV

In [2]:
random.seed(10)

In [3]:
train = pd.DataFrame.from_csv('data/train.csv')

In [4]:
test = pd.DataFrame.from_csv('data/test.csv')

In [36]:
le = preprocessing.LabelEncoder()

In [5]:
test.Survived = None

In [6]:
frames = [train, test]

In [7]:
## https://pandas.pydata.org/pandas-docs/stable/merging.html
combined = pd.concat(frames, keys=['train', 'test'])

In [34]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1309 entries, (train, 1) to (test, 1309)
Data columns (total 15 columns):
Age           1046 non-null float64
Cabin         295 non-null object
Embarked      1307 non-null object
Fare          1308 non-null float64
Name          1309 non-null object
Parch         1309 non-null int64
Pclass        1309 non-null int64
Sex           1309 non-null object
SibSp         1309 non-null int64
Survived      891 non-null float64
Ticket        1309 non-null object
Title         1309 non-null object
FamilySize    1309 non-null int64
Surname       1309 non-null object
FamilyId      1309 non-null object
dtypes: float64(3), int64(4), object(8)
memory usage: 167.5+ KB


In [8]:
def get_title(name):
    return name.split('.')[0].split(',')[1].strip(' \t\n\r')
def get_surname(name):
    return name.split(',')[0].strip(' \t\n\r')

In [9]:
combined['Title'] = combined['Name'].apply(get_title)

In [10]:
# https://www.kaggle.com/startupsci/titanic-data-science-solutions
combined['Title'] = combined['Title'].replace('Mme', 'Mlle')
combined['Title'] = combined['Title'].replace('Capt', 'Sir').replace('Don', 'Sir').replace('Major', 'Sir')
combined['Title'] = combined['Title'].replace('Dona', 'Lady').replace('the Countess', 'Lady').replace('Jonkheer', 'Lady')

In [11]:
combined['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Sir', 'Rev', 'Dr', 'Mlle', 'Ms',
       'Lady', 'Col'], dtype=object)

In [12]:
combined['FamilySize'] = combined['SibSp'] + combined['Parch'] + 1

In [13]:
combined['Surname'] = combined['Name'].apply(get_surname)

In [14]:
def create_id(surname, size):
    return str(size).strip(' \t\n\r') + surname

In [15]:
combined['FamilyId'] = combined['FamilySize'].map(str) + combined['Surname']

In [16]:
combined_ordered = combined.sort_values(['FamilyId'], ascending=True)

In [17]:
combined_ordered['Age'] = combined_ordered['Age'].interpolate(method='linear', order=2)

In [18]:
combined_ordered['Fare'] = combined_ordered['Fare'].interpolate(method='linear', order=2)

In [70]:
train_new = combined_ordered.loc['train']
test_new = combined_ordered.loc['test']

In [71]:
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Title', 'FamilySize', 'Embarked']

In [72]:
X_train_new = train_new[features].dropna()

In [73]:
y_train_new = train_new['Survived']

In [74]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=25)

In [54]:
train_t, test_t = train_test_split(test_new, test_size=0.33, random_state=42)

In [76]:
clf.fit(X_train_new, y_train_new)

ValueError: could not convert string to float: 'S'